In [1]:
import time
from datetime import timedelta

import html
import re

import os
import ntpath

import subprocess as sp

import numpy as np
import pandas as pd
import xml.etree.ElementTree as et

import dask
import dask.dataframe as dd
import dask.bag as bd
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

## Enter full path to xml file

In [2]:
xml_file = 'pmd1/pmdxmlreports_xml/pmd*.xml'

## Read xml file in block sizes

In [3]:
## Read xml file in block sizes
bg = bd.read_text(xml_file, blocksize='10MB', linedelimiter='</error>\n')

In [4]:
bg

dask.bag<bag-from-delayed, npartitions=577>

In [5]:
#bg.take(1)

## Replace the xml header with empty string '<xml ...>' => ''

In [6]:
## Replace the xml header with empty string '<xml ...>' => ''
bg = bg.map(lambda line: re.sub(r'\s*<\s*\?xml.*\?>\n', '', line)) 

In [7]:
#bg.take(1)

## Replace the pmd header with empty string '<pmd  ...>' => ''

In [8]:
## Replace the xml header with empty string '<xml ...>' => ''
bg = bg.map(lambda line: re.sub(r'\s*<\s*pmd.*>\n', '', line)) 

In [9]:
#bg.take(1)

In [10]:
# replace all the '<file>' with '</file>' so that we can access it using xml.etree.ElementTree
bg = bg.map(lambda line: re.sub(r'\s*<\s*file.*>|.*/file>', '', line))

In [11]:
#bg.take(1)

### Drop wherever the violation tage occurs and the content within violation tag

In [12]:
# replace all the '<violation>' with '</violation>' so that we can access it using xml.etree.ElementTree
#bg = bg.map(lambda line: re.sub(r'\s*<\s*violation.*>|.*/violation>', '', line))
bg = bg.map(lambda line: re.sub(r'\s*<\s*violation.*>\n(.*?)\n</violation>', '', line))

In [13]:
bg.take(1)

('\n\n<error filename="/scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/pmd1/mvn_apps/my-javacodeanalysis-app0/src/main/java/myjavacodeanalysis/Code_10022570_10022548_382_1.java" msg="PMDException: Error while parsing /scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/pmd1/mvn_apps/my-javacodeanalysis-app0/src/main/java/myjavacodeanalysis/Code_10022570_10022548_382_1.java">\n<![CDATA[net.sourceforge.pmd.PMDException: Error while parsing /scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/pmd1/mvn_apps/my-javacodeanalysis-app0/src/main/java/myjavacodeanalysis/Code_10022570_10022548_382_1.java\n\tat net.sourceforge.pmd.SourceCodeProcessor.processSourceCodeWithoutCache(SourceCodeProcessor.java:124)\n\tat net.sourceforge.pmd.SourceCodeProcessor.processSourceC

### Drop wherever the \n occurs

In [14]:
# replace all the '<violation>' with '</violation>' so that we can access it using xml.etree.ElementTree
bg = bg.map(lambda line: re.sub(r'\n+', '', line))

In [15]:
bg.take(1)

('<error filename="/scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/pmd1/mvn_apps/my-javacodeanalysis-app0/src/main/java/myjavacodeanalysis/Code_10022570_10022548_382_1.java" msg="PMDException: Error while parsing /scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/pmd1/mvn_apps/my-javacodeanalysis-app0/src/main/java/myjavacodeanalysis/Code_10022570_10022548_382_1.java"><![CDATA[net.sourceforge.pmd.PMDException: Error while parsing /scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/pmd1/mvn_apps/my-javacodeanalysis-app0/src/main/java/myjavacodeanalysis/Code_10022570_10022548_382_1.java\tat net.sourceforge.pmd.SourceCodeProcessor.processSourceCodeWithoutCache(SourceCodeProcessor.java:124)\tat net.sourceforge.pmd.SourceCodeProcessor.processSourceCode(Source

### Filter out just the error

In [16]:
## Filter out just the rows
bg = bg.filter(lambda line: line.find('<error') >= 0)

In [17]:
bg.take(5)

('<error filename="/scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/pmd1/mvn_apps/my-javacodeanalysis-app0/src/main/java/myjavacodeanalysis/Code_10022570_10022548_382_1.java" msg="PMDException: Error while parsing /scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/pmd1/mvn_apps/my-javacodeanalysis-app0/src/main/java/myjavacodeanalysis/Code_10022570_10022548_382_1.java"><![CDATA[net.sourceforge.pmd.PMDException: Error while parsing /scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/pmd1/mvn_apps/my-javacodeanalysis-app0/src/main/java/myjavacodeanalysis/Code_10022570_10022548_382_1.java\tat net.sourceforge.pmd.SourceCodeProcessor.processSourceCodeWithoutCache(SourceCodeProcessor.java:124)\tat net.sourceforge.pmd.SourceCodeProcessor.processSourceCode(Source

### Use ElementTree to get all the attrbutes and text in xmlor html tags

In [18]:
# This will give us just the attributes in the violation tag: <violation ...>
#violations_rows_bd = violations_rows_bd.map(lambda row: et.fromstring(row).attrib)

In [19]:
# This will give us just the text enclosed in the violation tag: <violation ...>
#violations_rows_bd = violations_rows_bd.map(lambda row: {'text':et.fromstring(row).text})

- To get both the the attrbutes and text in the violation tag 
    - We use add_text_to_dict_attrb() to achieve that

In [20]:
# Convert text in tag to key/value
def update_dict_key_wt_text_as_value(tag, value):
    _str = ''
    if tag == 'error': 
        _str = value
   
    return _str

In [21]:
#https://stackoverflow.com/questions/43921237/access-nested-children-in-xml-file-parsed-with-elementtree
def content(root):
    _str = ''
    for child in root.iter():
        # update dict with text as values
        _str = update_dict_key_wt_text_as_value(child.tag, child.text)
        
    return _str

In [22]:
## Use ElementTree to get all the attrbutes in xml tags
#bg = bg.map(lambda row: et.fromstring(row).attrib)
bg = bg.map(lambda row: content(et.fromstring(row)))

In [23]:
bg.take(2)

('net.sourceforge.pmd.PMDException: Error while parsing /scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/pmd1/mvn_apps/my-javacodeanalysis-app0/src/main/java/myjavacodeanalysis/Code_10022570_10022548_382_1.java\tat net.sourceforge.pmd.SourceCodeProcessor.processSourceCodeWithoutCache(SourceCodeProcessor.java:124)\tat net.sourceforge.pmd.SourceCodeProcessor.processSourceCode(SourceCodeProcessor.java:100)\tat net.sourceforge.pmd.SourceCodeProcessor.processSourceCode(SourceCodeProcessor.java:62)\tat net.sourceforge.pmd.processor.PmdRunnable.call(PmdRunnable.java:85)\tat net.sourceforge.pmd.processor.PmdRunnable.call(PmdRunnable.java:29)\tat java.util.concurrent.FutureTask.run(FutureTask.java:266)\tat java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:511)\tat java.util.concurrent.FutureTask.run(FutureTask.java:266)\tat java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:11

## Start a Dask cluster using SLURM jobs as workers.

There are a couple of things we need to configure here:

disabling the mechanism to write on disk when workers run out of memory,
memory, CPUs, maximum time and number of workers per SLURM job,
dask folders for log files and workers data.
We recommend putting the log folder and workers data folders in your /nesi/nobackup/<project_code> folder, most indicated for temporary files (see NeSI File Systems and Quotas).

All of these options can be set in configuration files, see Dask configuration and Dask jobqueue configuration for more information.

In [24]:
#http://jobqueue.dask.org/en/latest/generated/dask_jobqueue.SLURMCluster.html
dask.config.set(
    {
        "distributed.worker.memory.target": False,  # avoid spilling to disk
        "distributed.worker.memory.spill": False,  # avoid spilling to disk
    }
)
cluster = SLURMCluster(
    cores=10, #cores=24, # we set each job to have 1 Worker, each using 10 cores (threads) and 8 GB of memory
    processes=2,
    memory="8GiB",
    walltime="0-30:30",# walltime="0-00:30",
    log_directory="../dask/logs",  # folder for SLURM logs for each worker
    local_directory="../dask",  # folder for workers data
)

In [25]:
#cluster.adapt(minimum_jobs=20, maximum_jobs=200)
cluster.adapt(minimum_jobs=10, maximum_jobs=200)
client = Client(cluster)
client

Client Scheduler: tcp://192.168.94.156:46074 Dashboard: http://192.168.94.156:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


## Flatten

In [26]:
# you do not need to flatten
# bg = bg.flatten()

## Convert Bags to Dataframes

In [27]:
#convert from bags to dataframe
#ddf = bg.to_dataframe(meta={'name':'str', 'line':'int', 'severity':'str', 'message':'str', 'source':'str'})
ddf = bg.to_dataframe()

In [28]:
ddf

,0
npartitions=577,
,object
,...
...,...
,...
,...


In [29]:
ddf.columns

RangeIndex(start=0, stop=1, step=1)

In [30]:
ddf.head()

,0
0,net.sourceforge.pmd.PMDException: Error while ...
1,net.sourceforge.pmd.PMDException: Error while ...
2,net.sourceforge.pmd.PMDException: Error while ...
3,net.sourceforge.pmd.PMDException: Error while ...
4,net.sourceforge.pmd.PMDException: Error while ...


In [31]:
ddf.tail()

,0
2480,net.sourceforge.pmd.PMDException: Error while ...
2481,net.sourceforge.pmd.PMDException: Error while ...
2482,net.sourceforge.pmd.PMDException: Error while ...
2483,net.sourceforge.pmd.PMDException: Error while ...
2484,net.sourceforge.pmd.PMDException: Error while ...


In [32]:
# 1005519 errors
ddf.shape[0].compute()

1005519

In [34]:
# Save to a CSV file
_ = ddf.to_csv('pmdcodesnippeterrors_csv1/pmdCodeSnippetErrors*.csv', sep=',', index=False)

KilledWorker: ("('bag-from-delayed-decode-filter-lambda-read-block-lambda-lambda-to_dataframe-0e69b25cf1593fd5e7905756c7ad2198', 455)", <Worker 'tcp://10.65.4.42:37276', name: 347-1, memory: 0, processing: 10>)